# Azure OpenAI - AlphaCentauri Project
## Step 1: Setting up your Azure & Langchain
based on https://learn.deeplearning.ai/courses/building-your-own-database-agent

In [1]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)


True

In [ ]:

from openai import AzureOpenAI

# Azure OpenAI Configuration (CORRECT endpoint from Azure Portal)
#endpoint = "https://js-alphacentauri-resource.cognitiveservices.azure.com/"
endpoint ="https://js-alphacentauri-resource.openai.azure.com/"
deployment = "gpt-4.1-mini"
v_model = "gpt-4.1-mini"
api_version = "2025-03-01-preview"  # Updated to latest API version for Responses API support

# Get API key from environment
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

# Create Azure OpenAI client
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

print("✅ Azure OpenAI client configured")
print(f"Endpoint: {endpoint}")
print(f"Deployment: {deployment}")
print(f"API Version: {api_version}")


✅ Azure OpenAI client configured
Endpoint: https://js-alphacentauri-resource.openai.azure.com/
Deployment: gpt-4.1-mini
API Version: 2024-12-01-preview


In [3]:
# Test the connection
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Say hello and tell me you're working! Tell me not to worry, everything will be fine.",
        }
    ],
    model=deployment
)

print(response.choices[0].message.content)


Hello! I'm working right now, so no need to worry—everything will be just fine! If you need any assistance, I'm here to help.


### LangChain Integration


In [4]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Create LangChain model
model = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key
)

# Test with LangChain
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Translate 'Hello, how are you?' to French and Spanish.")
]

response = model.invoke(messages)
print(response.content)


Sure!

- French: Bonjour, comment ça va ?
- Spanish: Hola, ¿cómo estás?


### Streaming Response Example


In [5]:
v_messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Translate this sentence from English "
    "to French and Spanish. I like red cars and "
    "blue houses, but my dog is yellow.")
]   

## alternative way to stream the response
    # messages=[
    #     {
    #         "role": "system",
    #         "content": "You are a helpful assistant.",
    #     },
    #     {
    #         "role": "user",
    #         "content": "Translate this sentence from English "
    # "to French and Spanish. I like red cars and "
    # "blue houses, but my dog is yellow.",
    #     }
    # ],

# Stream the output for better user experience
response = client.chat.completions.create(
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "Translate this sentence from English "
    "to French, West Vlaams and Spanish. I like red cars and "
    "blue houses, but my dog is yellow.",
        }
    ],
    model=deployment,
)

print("Streaming response:")
for update in response:
    if update.choices:
        print(update.choices[0].delta.content or "", end="")


Streaming response:
Sure! Here is the translation of the sentence "I like red cars and blue houses, but my dog is yellow." in French, West Vlaams, and Spanish:

French:  
J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.

West Vlaams:  
'k Vin rode mûters en blauwe huzen leuk, moar mien hoond is geel.

Spanish:  
Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.

### 🔍 Debug: Verify Deployment Name


In [6]:
model.invoke((v_messages))

AIMessage(content="Sure! Here is the translation:\n\nFrench: J'aime les voitures rouges et les maisons bleues, mais mon chien est jaune.  \nSpanish: Me gustan los coches rojos y las casas azules, pero mi perro es amarillo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 41, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CXWCoCeOWHhXGQvJKK8gbHLXdToQB', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, '

## Step 2: Interacting with CSV Data

In [7]:
# Create LangChain model
model = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key
)


### Load the dataset
- baseline model -->
- could use 
  a) fine tuning for sql tasks 
  b) RAG (database as a source)
    - use Langchain agents to connect to SQL Database or CSV file
    - via Azure OpenAI Assistants API (function calling + code interpreter) (stateful management + short term memory)
    - via Azure OpenAI Fucntion Calling: to perform tasks based on your questions
    - Vai Native Database API

In [8]:
#read in the data from the csv file
#df = pd.read_csv("./data/synthetic_sales_data.csv").fillna(value = 0)
df = pd.read_csv("./data/synthetic_sales_data.csv", sep=";").fillna(value=0)

In [9]:
# Check if the file is read and display the first few rows with headers
print("First 5 rows of the dataset:")
print("=" * 80)
display(df.head())

# Alternative: use this if display() doesn't work
# print(df.head().to_string())

First 5 rows of the dataset:


,Region,ProductLine,Month,Revenue,UnitsSold,DiscountPercent,Cost,Profit,ProfitMargin,KPI_01,...,KPI_31,KPI_32,KPI_33,KPI_34,KPI_35,KPI_36,KPI_37,KPI_38,KPI_39,KPI_40
0,East,Clothing,2023-01-01 00:00:00,181890,69,"0,28","122553,98","59336,02","0,33","104,4",...,"100,97","86,57","118,95","95,2","104,3","116,46","105,26","82,91","117,61","90,59"
1,East,Clothing,2023-02-01 00:00:00,316208,952,"0,15","205913,08","110294,92","0,35","107,2",...,"83,42","95,96","83,36","112,95","109,02","127,5","96,35","112,5","105,15","114,32"
2,East,Clothing,2023-04-01 00:00:00,483125,1021,"0,17","400566,74","82558,26","0,17","102,56",...,"112,17","99,13","71,95","106,23","71,27","108,21","121,33","101,07","88,5","75,74"
3,East,Clothing,2023-10-01 00:00:00,476960,1856,"0,13","311779,56","165180,44","0,35","57,08",...,"110,58","124,64","92,32","100,18","108,17","88,18","123,39","101,25","84,87","65,21"
4,East,Electronics,2023-04-01 00:00:00,432515,177,"0,02","301553,9","130961,1","0,3","128,72",...,"92,89","90,96","109,71","90,64","97,44","81,56","89,29","113,26","124,89","96,62"


In [10]:
# Show detailed information about the dataset structure
print("📊 Dataset Information:")
print("-" * 80)
print(f"Total Rows: {len(df)}")
print(f"Total Columns: {len(df.columns)}")
print("\nColumn Names and Data Types:")
print("-" * 80)
for i, (col, dtype) in enumerate(zip(df.columns, df.dtypes), 1):
    print(f"{i:2d}. {col:30s} | Type: {dtype}")
print("-" * 80)


📊 Dataset Information:
--------------------------------------------------------------------------------
Total Rows: 100
Total Columns: 49

Column Names and Data Types:
--------------------------------------------------------------------------------
 1. Region                         | Type: object
 2. ProductLine                    | Type: object
 3. Month                          | Type: object
 4. Revenue                        | Type: int64
 5. UnitsSold                      | Type: int64
 6. DiscountPercent                | Type: object
 7. Cost                           | Type: object
 8. Profit                         | Type: object
 9. ProfitMargin                   | Type: object
10. KPI_01                         | Type: object
11. KPI_02                         | Type: object
12. KPI_03                         | Type: object
13. KPI_04                         | Type: object
14. KPI_05                         | Type: object
15. KPI_06                         | Type: object
16.

#### <span style="color: red">SOME ATTENTION REQUIRED</span> 
create_pandas_dataframe_agent() maak een “agent” die:
- Je dataframe (df) kent
- Je prompt interpreteert (“How many rows are there?”)
- Daarvoor zelf Python-code schrijft
- En die code uitvoert in een “Python REPL” (read–eval–print-loop) in jouw proces.
De “REPL” is in feite een mini-Python-console binnen jouw proces.
En de LLM (of een kwaadaardige prompt) kan daarin eender welke code uitvoeren.
Niet alleen veilige dingen zoals len(df), maar ook bijvoorbeeld:

--> gezien we controle hebben over de data hier kunnen we **allow_dangerous_code=True** toevoegen, MAAR GEBRUIK DIT NIET IN PRODUCTIE!


In [11]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm=model,
    df=df,
    verbose=True, 
    allow_dangerous_code=True,
    max_iterations=30,  # Increase from default 15
    max_execution_time=300  # 5 minutes timeout
)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe `df`, I can use the `len()` function or `df.shape[0]`.

Action: python_repl_ast
Action Input: len(df)100Thought: The dataframe `df` has 100 rows based on the length obtained.

Final Answer: There are 100 rows in the dataframe `df`.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 100 rows in the dataframe `df`.'}

In [12]:
# agent.invoke("What can you tell me about the data?")
# agent.invoke("List me all the diffrent productlines and their individual summed uprevenue")
agent.invoke("List per region which productline sold most units in total and how many units were sold.")




> Entering new AgentExecutor chain...
Thought: I need to group the dataframe by "Region" and "ProductLine", then sum the "UnitsSold" for each group. After that, I will find the product line with the highest total units sold per region.

Action: python_repl_ast
Action Input: df['UnitsSold'] = df['UnitsSold'].astype(int)  # Ensure UnitsSold is integer for aggregation
unit_sales_by_region_product = df.groupby(['Region', 'ProductLine'])['UnitsSold'].sum()
max_units_sold_per_region = unit_sales_by_region_product.groupby(level=0).idxmax()
result = [(region, product, unit_sales_by_region_product[idx]) for region, product in max_units_sold_per_region]
resultNameError: name 'idx' is not definedThought: I made a mistake by using the undefined variable `idx` in the list comprehension. I should use the elements directly from `max_units_sold_per_region`, which are index tuples, to access the sums from `unit_sales_by_region_product`.

Action: python_repl_ast
Action Input: result = [(region, produc

{'input': 'List per region which productline sold most units in total and how many units were sold.',
 'output': '- East: Furniture sold most units with a total of 7533 units\n- North: Toys sold most units with a total of 5778 units\n- South: Groceries sold most units with a total of 9485 units\n- West: Toys sold most units with a total of 9022 units'}

In [13]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

#QUESTION = "How may patients were hospitalized during July 2020" 
#"in Texas, and nationwide as the total of all states?"
SIMPLE_PROMPT = """
How many patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states?
Use the hospitalizedIncrease column.

Instructions:
1. Filter the data for July 2020 (dates between 2020-07-01 and 2020-07-31)
2. Sum the hospitalizedIncrease column for Texas
3. Sum the hospitalizedIncrease column for all states
4. Provide both answers with a clear explanation
"""

SIMPLE_PROMPT_SALES = """
How many toys units are sold in the month of July, regardless the year?
"""

QUESTION = "How may patients were hospitalized during July 2020" \
"in Texas, and nationwide as the total of all states?" \
"Use the hospitalizedIncrease column" 


output = agent.invoke(CSV_PROMPT_PREFIX + SIMPLE_PROMPT_SALES + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Question: 
How many toys units are sold in the month of July, regardless the year?

Thought: I need to first set the pandas display options to show all columns, then get the column names to understand the dataframe structure thoroughly. After that, filter rows where the ProductLine is "Toys" and the month is July (month=7) regardless of the year, then sum the UnitsSold for those rows.

Action: python_repl_ast
Action Input: |
    import pandas as pd
    
    # Show all columns
    pd.set_option('display.max_columns', None)
    cols = df.columns.tolist()
    colsSyntaxError: invalid syntax (<unknown>, line 1)Thought: The previous input gave a syntax error because the input might have taken multiple lines or there was an attempt to run multi-line code in one go incorrectly. I will run commands one by one to avoid syntax errors.

First, set the pandas display options to show all columns. Then get the column names.

Action: python_repl_ast
Action Inpu

In [14]:
# Alternative: Use the complex prompt with reflection (now with increased iteration limit)
# Uncomment the line below to try the more thorough approach with validation
# agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)


In [15]:
# Access the output from the dictionary
print(output['output'])

# Or display it in markdown for better formatting
# display(Markdown(output['output']))

**

**6258 units** of toys were sold in the month of July, regardless of the year.


## Step 3: Connecting to a SQL Database

In [16]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase


In [17]:
import urllib.request

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download the file using Python (works on Windows, Mac, Linux)
# url = "https://covidtracking.com/data/download/all-states-history.csv"
# file_path = "./data/all-states-history.csv"

# print(f"Downloading from {url}...")
# urllib.request.urlretrieve(url, file_path)
# print(f"✅ File downloaded to {file_path}")

# # Load the data
# df = pd.read_csv(file_path).fillna(value=0)
# print(f"✅ Loaded {len(df)} rows of data")
df.head()

,Region,ProductLine,Month,Revenue,UnitsSold,DiscountPercent,Cost,Profit,ProfitMargin,KPI_01,KPI_02,KPI_03,KPI_04,KPI_05,KPI_06,KPI_07,KPI_08,KPI_09,KPI_10,KPI_11,KPI_12,KPI_13,KPI_14,KPI_15,KPI_16,KPI_17,KPI_18,KPI_19,KPI_20,KPI_21,KPI_22,KPI_23,KPI_24,KPI_25,KPI_26,KPI_27,KPI_28,KPI_29,KPI_30,KPI_31,KPI_32,KPI_33,KPI_34,KPI_35,KPI_36,KPI_37,KPI_38,KPI_39,KPI_40,Month_num
0,East,Clothing,2023-01-01,181890,69,"0,28","122553,98","59336,02","0,33","104,4","100,26","84,07","99,84","115,72","108,62","98,34","115,8","113,63","70,08","106,3","107,12","72,85","114,91","93,97","105,16","111,21","87,6","118,52","79,26","111,74","108,97","127,39","73,58","131,22","84,08","120,64","77,37","76,65","63,46","100,97","86,57","118,95","95,2","104,3","116,46","105,26","82,91","117,61","90,59",1
1,East,Clothing,2023-02-01,316208,952,"0,15","205913,08","110294,92","0,35","107,2","104,32","78,91","122,06","134,75","116,68","162,1","124,46","134,17","88,75","117,2","73,45","83,93","70,57","116,18","101,7","80,71","81,31","133,67","145,8","105,28","89,83","84,14","132,17","48,52","87,11","91,2","96,33","123,34","116,36","83,42","95,96","83,36","112,95","109,02","127,5","96,35","112,5","105,15","114,32",2
2,East,Clothing,2023-04-01,483125,1021,"0,17","400566,74","82558,26","0,17","102,56","141,34","68,89","82,02","80,93","118,84","74,46","126,76","66,26","118,75",93,"108,58","77,16","124,77","106,34","106,54","58,53","103,76","71,63","119,35","65,62","83,06","141,55","104,86","110,39","116,79","131,92","116,83","105,27","106,16","112,17","99,13","71,95","106,23","71,27","108,21","121,33","101,07","88,5","75,74",4
3,East,Clothing,2023-10-01,476960,1856,"0,13","311779,56","165180,44","0,35","57,08","116,4","145,13","104,72","55,09","139,26","100,34","126,38","124,98","69,35","96,43","102,58","107,8","111,6","94,55","97,87","99,93","66,22","127,34","89,94","92,07","114,52",66,"91,12","107,19","81,75","90,23","117,25","79,8","82,89","110,58","124,64","92,32","100,18","108,17","88,18","123,39","101,25","84,87","65,21",10
4,East,Electronics,2023-04-01,432515,177,"0,02","301553,9","130961,1","0,3","128,72","104,47","99,22","103,45","92,77","98,1","111,19","82,11","123,05","124,37","94,86","81,17","128,59","86,49","97,7","76,43","113,44","87,3","126,34","146,01","87,44","88,7","100,16","89,44","97,98","112,66","92,87","72,72","148,59","121,93","92,89","90,96","109,71","90,64","97,44","81,56","89,29","113,26","124,89","96,62",4


In [18]:
from sqlalchemy import create_engine

# Path to your SQLite database file
# database_file_path = "./data/test.db"
database_file_path = "./data/sales_db.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
# file_url = "./data/all-states-history.csv"
# df = pd.read_csv(file_url).fillna(value = 0)
# df.to_sql(
#     'all_states_history',
#     con=engine,
#     if_exists='replace',
#     index=False
# )
df.to_sql(
    'sales_db',
    con=engine,
    if_exists='replace',
    index=False
)



100

In [19]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [20]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT TOP (10) [death]
FROM covidtracking 
WHERE state = 'TX' AND date LIKE '2020%'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

"""

In [47]:
# Create LangChain model
model2 = AzureChatOpenAI(
    openai_api_version=api_version,
    azure_deployment=deployment,
    azure_endpoint=endpoint,
    api_key=subscription_key,
    temperature = 0,
    max_tokens=2000  # Increased to allow proper agent format responses with SQL queries and explanations
)
db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=model2)

In [48]:
QUESTION = """How much money did we make?
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model2,
    toolkit=toolkit,
    top_k=30,
    verbose=True,
    handle_parsing_errors=True   # don't crash on parsing errors
)

In [49]:
output_2 = agent_executor_SQL.invoke({"input": QUESTION})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: sales_dbAction: sql_db_schema
Action Input: sales_db
CREATE TABLE sales_db (
	"Region" TEXT, 
	"ProductLine" TEXT, 
	"Month" DATETIME, 
	"Revenue" BIGINT, 
	"UnitsSold" BIGINT, 
	"DiscountPercent" TEXT, 
	"Cost" TEXT, 
	"Profit" TEXT, 
	"ProfitMargin" TEXT, 
	"KPI_01" TEXT, 
	"KPI_02" TEXT, 
	"KPI_03" TEXT, 
	"KPI_04" TEXT, 
	"KPI_05" TEXT, 
	"KPI_06" TEXT, 
	"KPI_07" TEXT, 
	"KPI_08" TEXT, 
	"KPI_09" TEXT, 
	"KPI_10" TEXT, 
	"KPI_11" TEXT, 
	"KPI_12" TEXT, 
	"KPI_13" TEXT, 
	"KPI_14" TEXT, 
	"KPI_15" TEXT, 
	"KPI_16" TEXT, 
	"KPI_17" TEXT, 
	"KPI_18" TEXT, 
	"KPI_19" TEXT, 
	"KPI_20" TEXT, 
	"KPI_21" TEXT, 
	"KPI_22" TEXT, 
	"KPI_23" TEXT, 
	"KPI_24" TEXT, 
	"KPI_25" TEXT, 
	"KPI_26" TEXT, 
	"KPI_27" TEXT, 
	"KPI_28" TEXT, 
	"KPI_29" TEXT, 
	"KPI_30" TEXT, 
	"KPI_31" TEXT, 
	"KPI_32" TEXT, 
	"KPI_33" TEXT, 
	"KPI_34" TEXT, 
	"KPI_35" TEXT, 
	"KPI_36" TEXT, 
	"KPI_37" TEXT, 
	"KPI_38" TEXT, 
	"KPI_39"

In [50]:

# Or display it in markdown for better formatting
display(Markdown(output_2['input']))
display(Markdown(output_2['output']))

How much money did we make?


The total money made (total revenue) is 25,762,666.

Explanation:
I found a table named `sales_db` which contains a column `Revenue` that represents the money made. I wrote a query to sum up all the values in the `Revenue` column to get the total revenue. The query I used was:

```sql
SELECT SUM(Revenue) AS TotalRevenue FROM sales_db LIMIT 30
```

The result of this query showed that the total revenue is 25,762,666.

In [51]:
QUESTION = """Which product line had the highest average profit margin in the West region during 2023, and how much higher was it compared to the lowest one?
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model2,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [52]:
output_challenge = agent_executor_SQL.invoke({"input": QUESTION})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: sales_dbAction: sql_db_schema
Action Input: sales_db
CREATE TABLE sales_db (
	"Region" TEXT, 
	"ProductLine" TEXT, 
	"Month" DATETIME, 
	"Revenue" BIGINT, 
	"UnitsSold" BIGINT, 
	"DiscountPercent" TEXT, 
	"Cost" TEXT, 
	"Profit" TEXT, 
	"ProfitMargin" TEXT, 
	"KPI_01" TEXT, 
	"KPI_02" TEXT, 
	"KPI_03" TEXT, 
	"KPI_04" TEXT, 
	"KPI_05" TEXT, 
	"KPI_06" TEXT, 
	"KPI_07" TEXT, 
	"KPI_08" TEXT, 
	"KPI_09" TEXT, 
	"KPI_10" TEXT, 
	"KPI_11" TEXT, 
	"KPI_12" TEXT, 
	"KPI_13" TEXT, 
	"KPI_14" TEXT, 
	"KPI_15" TEXT, 
	"KPI_16" TEXT, 
	"KPI_17" TEXT, 
	"KPI_18" TEXT, 
	"KPI_19" TEXT, 
	"KPI_20" TEXT, 
	"KPI_21" TEXT, 
	"KPI_22" TEXT, 
	"KPI_23" TEXT, 
	"KPI_24" TEXT, 
	"KPI_25" TEXT, 
	"KPI_26" TEXT, 
	"KPI_27" TEXT, 
	"KPI_28" TEXT, 
	"KPI_29" TEXT, 
	"KPI_30" TEXT, 
	"KPI_31" TEXT, 
	"KPI_32" TEXT, 
	"KPI_33" TEXT, 
	"KPI_34" TEXT, 
	"KPI_35" TEXT, 
	"KPI_36" TEXT, 
	"KPI_37" TEXT, 
	"KPI_38" TEXT, 
	"KPI_39"

In [53]:
display(Markdown(output_challenge['input']))
display(Markdown(output_challenge['output']))

Which product line had the highest average profit margin in the West region during 2023, and how much higher was it compared to the lowest one?


The "Toys" product line had the highest average profit margin in the West region during 2023, which was about 0.156 (15.6 percentage points) higher than the lowest one, "Clothing."

Explanation:
I queried the `sales_db` table for the average profit margin by product line in the West region for the year 2023. I converted the `ProfitMargin` field from text with commas to a float for accurate averaging. Then I ordered the results by average profit margin descending to find the highest and lowest values. The difference between these two values was calculated to answer how much higher the highest was compared to the lowest.

The SQL query used was:

```sql
SELECT ProductLine, AVG(CAST(REPLACE(ProfitMargin, ',', '.') AS FLOAT)) AS AvgProfitMargin
FROM sales_db
WHERE Region = 'West' AND strftime('%Y', Month) = '2023'
GROUP BY ProductLine
ORDER BY AvgProfitMargin DESC
LIMIT 30
```

## STEP 4. Azure OpenAI Function Calling
What is the additional values
- provide specific isntructions for finding information 
- prioritzie queries for precise reulst and desired formats
- more control 

In [54]:

import json

## example of fucntion 
def get_current_weather(location, unit="Celsius"):
    """Get the current weather in a given location. 
    The default unit when not specified is Celsius"""
    if "merelbeke" in location.lower():
        return json.dumps(
            {"location": "Merelbeke", "country":"Belgium", "temperature": "20", "unit": unit}
        )
    elif "antwerpen" in location.lower():
        return json.dumps(
            {"location": "Antwerpen", "country":"Belgium", "temperature": "25", "unit": unit}
        )
    elif "las vegas" in location.lower():
        return json.dumps(
            {"location": "Las Vegas", "country":"USA", "temperature": "35", "unit": unit}
        )
    else:
        return json.dumps(
            {"location": location, "country":"unknown", "temperature": "unknown", "unit":  unit}
        )

get_current_weather("Merelbeke")

'{"location": "Merelbeke", "country": "Belgium", "temperature": "20", "unit": "Celsius"}'

In [55]:

# user prompt
weather_messages = [
    {"role": "user",
     "content": """What's the weather like in Merelbeke,
                   Antwerpen, and Las Vegass?"""
    }
]

#tool definition

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": """Get the current weather in a given
                              location.The default unit when not
                              specified is Celsius""",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": """The city and state,
                                        e.g. San Francisco, CA""",
                    },
                    "unit": {
                        "type": "string",
                        "default":"Celsius",
                        "enum": [ "Fahrenheit", "Celsius"],
                        "description": """The messuring unit for
                                          the temperature.
                                          If not explicitly specified
                                          the default unit is 
                                          Celsius"""
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [56]:
#defining the OpenAI clietn class
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


#just call the class
response = client.chat.completions.create(
    model=v_model,
    messages=weather_messages,
    tools=tools,
    tool_choice="auto", 
)

response_message = response.choices[0].message
print ("Response message: \n" , response_message, "\n\n" )

tool_calls = response_message.tool_calls

print ("TOOLS CALLS: \n" , tool_calls , "\n\n" )

available_functions = {
    "get_current_weather": get_current_weather,
} 

answers = []

if tool_calls:
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit", "Celsius")  ## you can see from the function=Fuction that the only argument is location
        )
        answers.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
    print ("Answers: \n" , answers , "\n\n" ) 
    
   
    def print_arguments(obj):
        "Recursively find and print JSON arguments or content with key/value pairs."
        if isinstance(obj, list):
            for item in obj:
                print_arguments(item)
        elif isinstance(obj, dict):
            # if it looks like a serialized JSON string of arguments/content, try parsing it
            for key, val in obj.items():
                if key in ("arguments", "content") and isinstance(val, str) and val.strip().startswith("{"):
                    try:
                        parsed = json.loads(val)
                        for k, v in parsed.items():
                            if k == "unit":
                                print(f"{k}: {v} \n")
                                v = 1
                            else:
                                print(f"{k}: {v}")
                    except Exception:
                        pass
                else:
                    print_arguments(val)

print_arguments(answers)

Response message: 
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_fcx4Q0P4V2ZzmP1OkCxF1w6L', function=Function(arguments='{"location": "Merelbeke"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_y9Ogf86wyBnavteaEkUfG0TJ', function=Function(arguments='{"location": "Antwerpen"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_7WwSwbsRssoqwrn7lhT4QI7Q', function=Function(arguments='{"location": "Las Vegas"}', name='get_current_weather'), type='function')]) 


TOOLS CALLS: 
 [ChatCompletionMessageFunctionToolCall(id='call_fcx4Q0P4V2ZzmP1OkCxF1w6L', function=Function(arguments='{"location": "Merelbeke"}', name='get_current_weather'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_y9Ogf86wyBnavteaEkUfG0TJ', function=Function(arguments='{"location": "Ant

In [57]:
import numpy as np
from sqlalchemy import text

def Total_KPI_01(reg, prodline):
    try:
        query = f"""
        SELECT region, productline,  KPI_01
        FROM sales_db
        WHERE region = '{reg}' AND 
              productline = '{prodline}'
        group by region, productline;
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
            
        if not result.empty:
            return result.to_dict('records')
        else:
            return np.nan
        return result
    except Exception as e:
        print(e)
        return np.nan
    
    
def sum_profit_cost (reg, prodline):
    try:
        query = f"""
        SELECT region, productline, sum(profit), sum(cost)
        FROM sales_db
        where region = '{reg}'  AND
            productline = '{prodline}'
        group by region, productline;
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
            
        if not result.empty:
            return result.to_dict('records')
        else:
            return np.nan
        return result
    except Exception as e:
        print(e)
        return np.nan

In [58]:
print(Total_KPI_01("East","Toys"))

print(sum_profit_cost("East","Toys"))

[{'Region': 'East', 'ProductLine': 'Toys', 'KPI_01': '63,61'}]
[{'Region': 'East', 'ProductLine': 'Toys', 'sum(profit)': 406321.0, 'sum(cost)': 860659.0}]


In [59]:
sql_messages = [
    {"role": "user",
     "content": """ How much what the summed up KPI_01 for Region East and productline Toys?"""
    },
    {"role": "user",
     "content": """ What is the sum of the profits and costs for Region East and productline Clothing"""
    },
    
]

In [60]:
tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_total_kpi_01_for_region_productline",
            "description": """Retrieves the sum of the KPI_01 for a specified region and specified productline.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "reg": {
                        "type": "string",
                        "description": """The name of the region
                                          (e.g., 'East', 'West')."""
                    },
                    "prodline": {
                        "type": "string",
                        "description": """The name of the productline 
                                          (e.g. 'Toys','Clothing')."""
                    }
                },
                "required": ["reg", "prodline"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_sum_profit_cost_for_region_productline",
            "description": """Retrieves the sum of the profits and the costs for a specified region and specified productline
                                """,
            "parameters": {
                "type": "object",
                "properties": {
                    "reg": {
                        "type": "string",
                        "description": """The name of the region
                                          (e.g., 'East', 'West')."""
                    },
                    "prodline": {
                        "type": "string",
                        "description": """The name of the productline 
                                          (e.g. 'Toys','Clothing')."""
                    }
                },
                "required": ["reg", "prodline"]
            }
        }
    }
]

In [61]:
#just call the class
response = client.chat.completions.create(
    model=v_model,
    messages=sql_messages,
    tools=tools_sql,
    tool_choice="auto", 
)

response_sql_message = response.choices[0].message
tool_calls = response_sql_message.tool_calls

print (tool_calls)
sql_answers = []

available_functions = {
    "get_total_kpi_01_for_region_productline": Total_KPI_01,
    "get_sum_profit_cost_for_region_productline":sum_profit_cost
}  

if tool_calls:
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            reg=function_args.get("reg"),
            prodline=function_args.get("prodline"),
        )
        sql_answers.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            }
        ) 
    print ("Answers: \n" , sql_answers , "\n\n" ) 
    
import ast

for ans in sql_answers:
    content = ast.literal_eval(ans['content'])
    
    # handle both list and dict formats safely
    data = content[0] if isinstance(content, list) else content

    print(f"- Request: {ans['name']}")
    print(f"- Region: {data.get('Region', '')}")
    print(f"- ProdLine: {data.get('ProductLine', '')}")
    print(f"- KPI_01: {data.get('KPI_01', '').replace(',', '.')}")
    print(f"- Sum of profit: {data.get('sum(profit)', '')}")
    print(f"- Sum of Cost: {data.get('sum(cost)', '')}")

[ChatCompletionMessageFunctionToolCall(id='call_SMSAOA7Zd3Qt3HI2sO962iMi', function=Function(arguments='{"reg": "East", "prodline": "Toys"}', name='get_total_kpi_01_for_region_productline'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_EINdBbhcU7zkIIyADqhqdv2l', function=Function(arguments='{"reg": "East", "prodline": "Clothing"}', name='get_sum_profit_cost_for_region_productline'), type='function')]
Answers: 
 [{'tool_call_id': 'call_SMSAOA7Zd3Qt3HI2sO962iMi', 'role': 'tool', 'name': 'get_total_kpi_01_for_region_productline', 'content': "[{'Region': 'East', 'ProductLine': 'Toys', 'KPI_01': '63,61'}]"}, {'tool_call_id': 'call_EINdBbhcU7zkIIyADqhqdv2l', 'role': 'tool', 'name': 'get_sum_profit_cost_for_region_productline', 'content': "[{'Region': 'East', 'ProductLine': 'Clothing', 'sum(profit)': 417368.0, 'sum(cost)': 1040811.0}]"}] 


- Request: get_total_kpi_01_for_region_productline
- Region: East
- ProdLine: Toys
- KPI_01: 63.61
- Sum of profit: 
- Sum of Cost: 
-

In [62]:
second_response = client.chat.completions.create(
            model=v_model,
            messages=sql_messages,
        )
print (second_response)

ChatCompletion(id='chatcmpl-CXXFHgIAD1aMV45j4lE1qqTSrahiy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide the data or the table so I can calculate the summed up KPI_01 for Region East and productline Toys, as well as the sum of profits and costs for Region East and productline Clothing?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1762108655, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_3dcd5944f5', usage=CompletionUsage(completion_tokens=45, prompt_tokens=44, total_tokens=89, completion_tokens_

## Step 5: Leveraging Assistants API for SQL Database
### Assistants API
- behoudt conversatie context - statefull vs de chatcomplete niet
- stateful, historiek van interacties
- supporteert functions / tool calling
- Depreciation note from OpenAI --> responses API (mid 2026)
### Code Interpreter
- maakt het mogelijk voor de Assistants API om python code te genereren, uit te voeren en veranderen

Eigenschap	|  Assistants API  |  Responses API

Stateful (threads & runs)  |  ✅ Ja  |  ❌ Nee

Contextbeheer  |  Server-side bij OpenAI  |  Client-side door jou

Flexibiliteit  |  Minder  |  Veel groter

Complexiteit  |  Hoog  |  Lager



In [70]:
import Helper
from Helper import get_positive_cases_for_state_on_date
from Helper import get_hospitalized_increase_for_state_on_date

In [ ]:
# I) Create assistant
assistant = client.beta.assistants.create(
  instructions="""You are an assistant answering questions 
                  about a Covid dataset.""",
  model=v_model,  # Use the deployment name configured at the top (gpt-4.1-mini)
  tools=Helper.tools_sql)

# response = client.responses.create(
#     model=v_model,
#     instructions="You are an assistant answering questions about a Covid dataset.",
#     tools=Helper.tools_sql,
#     input=[
#         {"role": "user", "content": "What is the average infection rate by region?"}
#     ]
# )

# II) Create thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_qaXemZRyQyNPQZdo9zJcfvPu', created_at=1762116054, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\823333888.py:18: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


In [86]:
# III) Add message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message)

Message(id='msg_AcE6gwzaOhYULOD7dLUdfFUK', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1762116058, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_qaXemZRyQyNPQZdo9zJcfvPu')


C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\2942528415.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [87]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\708732456.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


{
  "data": [
    {
      "id": "msg_AcE6gwzaOhYULOD7dLUdfFUK",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1762116058,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_qaXemZRyQyNPQZdo9zJcfvPu"
    }
  ],
  "has_more": false,
  "object": "list",
  "first_id": "msg_AcE6gwzaOhYULOD7dLUdfFUK",
  "last_id": "msg_AcE6gwzaOhYULOD7dLUdfFUK"
}


In [88]:
# IV) Run assistant on thread

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\279332832.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [89]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    if (status=="requires_action"):
        available_functions = {
            "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
            "get_hospitalized_increase_for_state_on_date":get_hospitalized_increase_for_state_on_date
        }

        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                state_abbr=function_args.get("state_abbr"),
                specific_date=function_args.get("specific_date"),
            )
            print(function_response)
            print(tool_call.id)
            tool_outputs.append(
                { "tool_call_id": tool_call.id,
                 "output": str(function_response)
                }
            )

        run = client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs = tool_outputs
        )


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages)

C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\4045361210.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


Elapsed time: 0 minutes 5 seconds
Status: failed


C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\4045361210.py:49: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_AcE6gwzaOhYULOD7dLUdfFUK', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1762116058, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_qaXemZRyQyNPQZdo9zJcfvPu')], has_more=False, object='list', first_id='msg_AcE6gwzaOhYULOD7dLUdfFUK', last_id='msg_AcE6gwzaOhYULOD7dLUdfFUK')


In [90]:
# Check the error details
print("Run Status:", run.status)
print("\nFull Run Object:")
print(run)

if hasattr(run, 'last_error') and run.last_error:
    print("\n❌ ERROR DETAILS:")
    print(f"Error Code: {run.last_error.code}")
    print(f"Error Message: {run.last_error.message}")


Run Status: failed

Full Run Object:
Run(id='run_K4wUPW9Gi7af6EQ10A3C5Up6', assistant_id='asst_tkKTdD2lRxGfZFdsZjsJI59y', cancelled_at=None, completed_at=None, created_at=1762116108, expires_at=None, failed_at=1762116108, incomplete_details=None, instructions='You are an assistant answering questions \n                  about a Covid dataset.', last_error=LastError(code='invalid_engine_error', message='No connection matching model: gpt-4-1106'), max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1762116108, status='failed', thread_id='thread_qaXemZRyQyNPQZdo9zJcfvPu', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_hospitalized_increase_for_state_on_date', description='Retrieves the daily increase in hospitalizations for a specific state on a specific date.', parameters={'type': 'object', 'properties': {'state_abbr': {'t

In [78]:
print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_tL2ZUeBBTfGOxnsIKQLPB0Eu",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1762115140,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_iXTN53reDMJ9wUM72OBoJaQ6"
    }
  ],
  "has_more": false,
  "object": "list",
  "first_id": "msg_tL2ZUeBBTfGOxnsIKQLPB0Eu",
  "last_id": "msg_tL2ZUeBBTfGOxnsIKQLPB0Eu"
}


###Code Interpreter

In [83]:
file = client.files.create(
  file=open("./data/all-states-history.csv", "rb"),
  purpose='assistants'
)
assistant = client.beta.assistants.create(
  instructions="""You are an assitant answering questions about
                  a Covid dataset.""",
  model="gpt-4-1106", 
  tool_resources={
        "code_interpreter": {"file_ids": [file.id]}
    } 
  )## new syntx
thread = client.beta.threads.create()
print(thread)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\4105053699.py:13: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\4105053699.py:15: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


Thread(id='thread_TVJnZZXeto603cNT3YO9CdSp', created_at=1762115731, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Message(id='msg_wctejPjZsYPjZb9awvmkmgcR', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1762115731, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_TVJnZZXeto603cNT3YO9CdSp')


C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\4105053699.py:22: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [84]:
status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

C:\Users\jansy\AppData\Local\Temp\ipykernel_37640\3504059192.py:18: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(


{
  "data": [
    {
      "id": "msg_wctejPjZsYPjZb9awvmkmgcR",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1762115731,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_TVJnZZXeto603cNT3YO9CdSp"
    }
  ],
  "has_more": false,
  "object": "list",
  "first_id": "msg_wctejPjZsYPjZb9awvmkmgcR",
  "last_id": "msg_wctejPjZsYPjZb9awvmkmgcR"
}
